In [1]:
%matplotlib inline

import csv
import cv2
import math
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from random import shuffle
from skimage import exposure
from skimage import transform 

from sklearn.model_selection import train_test_split

def load_sample_list(paths, test_size=0.2, ratio=1):
  
    samples = []
    for path in paths:
        with open(path+'/driving_log.csv') as csvfile:
            reader = csv.reader(csvfile)
            next(reader)
            n = 0
            for line in reader:
                if not (n%ratio):
                    for i in range(3):
                        s = line[i].strip().replace('\\','/')
                        s = s.split('/')[-1]
                        line[i] = path + '/IMG/' + s
                    samples.append(line)
                n += 1
    
    samples.pop(0)
    
    shuffle(samples)
    
    train_samples, validation_samples = train_test_split(samples, test_size=test_size) 
    
    return(train_samples, validation_samples)

def generate_samples(samples, batch_size=32, cameras=('l','r','c'), flip=False):
    
    expanded = []
    steering_adj = 0.2 
    
    if flip:
        flips = [False, True]
    else:
        flips = [False]        
    
    for sample in samples:
        for camera in cameras:
            for flip in flips:
                expanded.append((sample) + [camera, flip,])
    
    num_samples = len(expanded)
    
    while 1:
        shuffle(expanded)  
        for offset in range(0, num_samples, batch_size):
            x = []
            y = []            
            for batch_sample in expanded[offset:offset+batch_size]:
                center,left,right,steering,throttle,brake,speed,camera,flip = batch_sample
                steering, throttle, brake, speed = float(steering), float(throttle), float(brake), float(speed)
                               
                if camera == 'c':
                    image = plt.imread(center)
                    steering = 0
                    
                if camera == 'l':                        
                    image = plt.imread(left)
                    steering += steering_adj                    
                    
                if camera == 'r':
                    image = plt.imread(right)
                    steering -= steering_adj
                    
                if False:
                    
                    max_shift = 80
                    r = np.random.randint(low=-max_shift, high=max_shift+1)

                    if r<0:
                        steering += steering_adj*r/max_shift
                        image[:,r:,:] = image[:,:-r,:]

                    if r>0:  
                        r = -r
                        steering -= steering_adj*r/max_shift
                        image[:,:-r,:] = image[:,r:,:] 
                 
                if flip:        
                    
                    image = np.fliplr(image) 
                    steering = -steering
                    
                x.append(image)
                y.append(steering)
                
            yield(np.array(x),np.array(y))

In [2]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Convolution2D, Cropping2D, Dropout, BatchNormalization

directories = ['Track1Forward','Track1Recovery'] # ,'Track2Forward','Track2Recovery']
train, valid = load_sample_list(directories, ratio=2)

# compile and train the model using the generator function
train_generator = generate_samples(train, batch_size=32)
validation_generator = generate_samples(valid, batch_size=32)

rate = .5
    
model = Sequential()

model.add(Cropping2D(cropping=((70,25), (0,0)), input_shape=(160, 320, 3)))
model.add(BatchNormalization())

model.add(Convolution2D(24,5,5,subsample=(2,2),activation='elu', init='glorot_uniform', bias=False))
model.add(BatchNormalization())

model.add(Convolution2D(36,5,5,subsample=(2,2),activation='elu', init='glorot_uniform', bias=False))
model.add(BatchNormalization())

model.add(Convolution2D(48,5,5,subsample=(2,2),activation='elu', init='glorot_uniform', bias=False))
model.add(BatchNormalization())

model.add(Convolution2D(64,3,3,activation='elu', init='glorot_uniform', bias=False))
model.add(BatchNormalization())

model.add(Convolution2D(64,3,3,activation='elu', init='glorot_uniform', bias=False))
model.add(BatchNormalization())

model.add(Flatten())

bias = False
model.add(Dropout(rate))
model.add(Dense(1154, activation='elu', init='glorot_uniform', bias=bias))
model.add(BatchNormalization())

model.add(Dropout(rate))
model.add(Dense( 100, activation='elu',  init='glorot_uniform', bias=bias))
model.add(BatchNormalization())

model.add(Dropout(rate))          
model.add(Dense(  50, activation='elu', init='glorot_uniform', bias=bias))
model.add(BatchNormalization())

model.add(Dropout(rate))
model.add(Dense(  10, activation='elu', init='glorot_uniform', bias=bias))
model.add(BatchNormalization())

model.add(Dense(   1, activation='linear', init='glorot_uniform'))

model.compile(loss='mse', optimizer='adam')
expansion = 3
epochs = 4
model.fit_generator(train_generator, samples_per_epoch= \
            len(train)*expansion, validation_data=validation_generator, \
            nb_val_samples=len(valid)*expansion, nb_epoch=epochs)

model.save('model.h5')


Using TensorFlow backend.


Epoch 1/4
11391/11391 [==============================] - 26s - loss: 0.2110 - val_loss: 0.0320
Epoch 2/4
11391/11391 [==============================] - 22s - loss: 0.0467 - val_loss: 0.0242
Epoch 3/4
11391/11391 [==============================] - 21s - loss: 0.0351 - val_loss: 0.0230
Epoch 4/4
11391/11391 [==============================] - 22s - loss: 0.0301 - val_loss: 0.0211
